In [1]:
from pyspark.sql import SparkSession, functions as F
from pyspark import SparkConf
import configparser

In [2]:
conf = SparkConf() \
.setAppName("MongoDB Example") \
.setMaster("local[2]") \
.set("spark.jars.packages","org.mongodb.spark:mongo-spark-connector_2.12:3.0.1")

In [3]:
spark = SparkSession.builder \
.config(conf=conf) \
.getOrCreate()

:: loading settings :: url = jar:file:/opt/manual/spark-3.1.1-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/train/.ivy2/cache
The jars for the packages stored in: /home/train/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-faeb3a52-85a9-4d4c-b654-a7d6dc9222de;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
downloading https://repo1.maven.org/maven2/org/mongodb/spark/mongo-spark-connector_2.12/3.0.1/mongo-spark-connector_2.12-3.0.1.jar ...
	[SUCCESSFUL ] org.mongodb.spark#mongo-spark-connector_2.12;3.0.1!mongo-spark-connector_2.12.jar (430ms)
downloading https://repo1.maven.org/maven2/org/mongodb/mongodb-driver-sync/4.0.5/mongodb-driver-sync-4.0.5.jar ...
	[SUCCESSFUL ] org.mongodb#mongodb-driver-sync;4.0.5!mongodb-driver-sync.jar (169ms)
downloading https://repo1.maven.org/

In [4]:
 ! wget -O ~/datasets/Churn_Modelling.csv \
 https://raw.githubusercontent.com/erkansirin78/datasets/master/Churn_Modelling.csv

--2023-03-27 17:22:19--  https://raw.githubusercontent.com/erkansirin78/datasets/master/Churn_Modelling.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 674856 (659K) [text/plain]
Saving to: ‘/home/train/datasets/Churn_Modelling.csv’

100%[======================================>] 674,856     1.14MB/s   in 0.6s   

2023-03-27 17:22:20 (1.14 MB/s) - ‘/home/train/datasets/Churn_Modelling.csv’ saved [674856/674856]



In [5]:
df = spark.read \
.option("inferSchema",True) \
.option("header", True) \
.csv("file:///home/train/datasets/Churn_Modelling.csv")

In [6]:
df.limit(5).toPandas()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.printSchema()

root
 |-- RowNumber: integer (nullable = true)
 |-- CustomerId: integer (nullable = true)
 |-- Surname: string (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- Geography: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tenure: integer (nullable = true)
 |-- Balance: double (nullable = true)
 |-- NumOfProducts: integer (nullable = true)
 |-- HasCrCard: integer (nullable = true)
 |-- IsActiveMember: integer (nullable = true)
 |-- EstimatedSalary: double (nullable = true)
 |-- Exited: integer (nullable = true)



In [8]:
config = configparser.RawConfigParser()

config.read('./db_conn')
user_name = config.get('DB', 'user_name')
password = config.get('DB', 'password')
db_ip = config.get('DB', 'db_ip')
port = config.get('DB', 'port')
auth_database = config.get('DB', 'auth_database')

In [10]:
df.count()

10000

In [9]:
df.write.format("mongo").mode("append") \
.option("uri",f"mongodb://{user_name}:{password}@{db_ip}:{port}/?authSource={auth_database}") \
.option("database", "test") \
.option("collection", "churn") \
.save()

In [11]:
df_mongo = spark.read.format("mongo") \
.option("uri",f"mongodb://{user_name}:{password}@{db_ip}:{port}/?authSource={auth_database}") \
.option("database", "test") \
.option("collection", "churn") \
.load()

In [13]:
df_mongo.count()

10000

In [14]:
df_mongo.limit(5).toPandas()

,Age,Balance,CreditScore,CustomerId,EstimatedSalary,Exited,Gender,Geography,HasCrCard,IsActiveMember,NumOfProducts,RowNumber,Surname,Tenure,_id
0,42,0.00,619,15634602,101348.88,1,Female,France,1,1,1,1,Hargrave,2,"(6421a6a9eb09a016bb6ccc9a,)"
1,41,83807.86,608,15647311,112542.58,0,Female,Spain,0,1,1,2,Hill,1,"(6421a6a9eb09a016bb6ccc9b,)"
2,42,159660.80,502,15619304,113931.57,1,Female,France,1,0,3,3,Onio,8,"(6421a6a9eb09a016bb6ccc9c,)"
3,39,0.00,699,15701354,93826.63,0,Female,France,0,0,2,4,Boni,1,"(6421a6a9eb09a016bb6ccc9d,)"
4,43,125510.82,850,15737888,79084.10,0,Female,Spain,1,1,1,5,Mitchell,2,"(6421a6a9eb09a016bb6ccc9e,)"


In [15]:
df_mongo.createOrReplaceTempView("churn_tbl")

In [16]:
spark.sql("""
SELECT Age, Balance FROM churn_tbl LIMIT 5
""").show()

+---+---------+
|Age|  Balance|
+---+---------+
| 42|      0.0|
| 41| 83807.86|
| 42| 159660.8|
| 39|      0.0|
| 43|125510.82|
+---+---------+



In [17]:
spark.sql("""

    SELECT Exited, COUNT(1) FROM churn_tbl 
    GROUP BY Exited

""").toPandas()

,Exited,count(1)
0,1,2037
1,0,7963
